In [1]:
!pip install transformers torch scikit-learn pandas flask


     ------------------------------------ 101.7/101.7 kB 983.8 kB/s eta 0:00:00
     -------------------------------------- 227.6/227.6 kB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 97.9/97.9 kB 2.8 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from flask import Flask, request, jsonify
from transformers import DistilBertTokenizer, DistilBertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity


c:\Users\Gunjan\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
data = pd.read_csv('disease_data.csv')

data['Symptoms'] = data['Symptoms'].str.lower()
data['Disease'] = data['Disease'].str.lower()


In [5]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

c:\Users\Gunjan\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state
    sentence_embedding = torch.mean(embeddings, dim=1)
    return sentence_embedding.squeeze().numpy()

data['Embedding'] = data['Symptoms'].apply(lambda x: get_embedding(x))


In [7]:
import numpy as np

def predict_disease(user_input):
    user_embedding = get_embedding(user_input.lower())
    
    embeddings = np.stack(data['Embedding'].values)
    similarities = cosine_similarity([user_embedding], embeddings)[0]
    
    closest_match_index = similarities.argmax()
    
    disease = data.iloc[closest_match_index]['Disease']
    solution = data.iloc[closest_match_index]['Solution']
    
    return disease, solution
